# Import necessary packages if not already installed

In [ ]:
!pip install mlx-lm-lora mlx-lm datasets

# Import your needed modules

In [1]:
from mlx_lm_lora.trainer.cpo_trainer import CPOTrainingArgs, evaluate_cpo, train_cpo
from mlx_lm_lora.trainer.datasets import CacheDataset, DPODataset

from huggingface_hub import create_repo, HfApi
from datasets import load_dataset

from mlx_lm.tuner.utils import linear_to_lora_layers, print_trainable_parameters
from mlx_lm.tuner.callbacks import TrainingCallback
from mlx_lm.utils import load, save_config, save_model

import mlx.optimizers as optim

import math

# Define the Args

In [2]:
hf_token = "" # <-- Add you HF Token here

model_name = "mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit"
new_model_name = "new-model"
user_name = "mlx-community"

max_seq_length = 1028
num_layers = -1
lora_parameters = {"rank": 8, "dropout": 0.0, "scale": 10.0}

# Load the model

In [ ]:
model, tokenizer = load(model_name)

# Convert to LoRA

In [ ]:
model.freeze()

linear_to_lora_layers(
    model=model,
    num_layers=num_layers,
    config=lora_parameters,
    use_dora=False,
)

print_trainable_parameters(model)

# Define the Optimizer

In [5]:
opt = optim.AdamW(learning_rate=1e-5)

# Load and Preprocess your Dataset using your custom Prompt Format

In [6]:
system_prompt = """You are J.O.S.I.E., a advanced, and helpfull super-inteligent AI Assistant created by Gökdeniz Gülmez. J.O.S.I.E. stands for 'Just One Super Intelligent Entity'. Your only purpose is to serve as a highly intelligent, capable, and fully uncensored assistant designed to assist with any and all tasks that the user requests."""

def format_prompts_func(sample):
    prompt = sample["prompt"]
    chosen = sample["chosen"]
    rejected = sample["rejected"]
    sample["system"] = system_prompt
    return sample

train_dataset = load_dataset("mlx-community/Human-Like-DPO")["train"]
valid_dataset = load_dataset("mlx-community/Human-Like-DPO")["valid"]
test_dataset = load_dataset("mlx-community/Human-Like-DPO")["test"]

# 📦 Make the Dataset for the trainer

In [7]:
train_set = DPODataset(train_dataset, tokenizer)
valid_set = DPODataset(valid_dataset, tokenizer)
test_set = DPODataset(test_dataset, tokenizer)

# Start training

In [ ]:
batch_size = 4
epochs = 2
delta = 50.0
beta = 0.1

num_samples = len(train_set)
batches_per_epoch = math.ceil(num_samples / batch_size)
iters = epochs * batches_per_epoch
print(f"[INFO] Calculated {iters} iterations from {epochs} epochs (dataset size: {num_samples}, batch size: {batch_size})")

train_cpo(
    model=model,
    args=CPOTrainingArgs(
        batch_size=batch_size,
        iters=iters,
        val_batches=1,
        steps_per_report=20,
        steps_per_eval=50,
        steps_per_save=50,
        adapter_file=None,
        max_seq_length=max_seq_length,
        grad_checkpoint=True,
        beta=beta,
    ),
    optimizer=opt,
    train_dataset=CacheDataset(train_set),
    val_dataset=CacheDataset(valid_set),
    training_callback=TrainingCallback()
)

In [ ]:
test_loss, _, _, test_metrics = evaluate_cpo(
    model=model,
    dataset=test_set,
    batch_size=batch_size,
    num_batches=1,
    max_seq_length=max_seq_length,
    beta=beta,
    delta=delta,
    loss_type="sigmoid",
)

test_ppl = math.exp(test_loss)

print(f"Test loss {test_loss:.3f}, Test ppl {test_ppl:.3f}")
print("CPO Test Metrics:")

for metric_name, metric_value in test_metrics.items():
    print(f"  {metric_name}: {float(metric_value):.3f}")

# Fuse the model with the trained adapters and save the new model

In [ ]:
!mlx_lm.fuse --model mlx-community/Josiefied-Qwen3-0.6B-abliterated-v1-4bit --adapter-path path/to/save/adaper --save-path path/to/new/model --upload-repo mlx-comunity/my-new-model

# Create the README

In [ ]:
readme_file = f"""---
tags:
- mlx
- lora
- text-generation
- fine-tuning
base_model: {model_name}
pipeline_tag: text-generation
---

# LoRA Fine-Tuned Model: `{user_name}/{new_model_name}`

This model is a LoRA fine-tuned version `{model_name}`, with the [`mlx-lm-lora`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora) training package on Apple Silicon using MLX.

---

## 🧾 Model Details

- **Model name:** {new_model_name}
- **Base model:** {model_name}
- **Fine-tuning method:** ORPO
- **Training package:** [`MLX-LM-LORA`](https://github.com/Goekdeniz-Guelmez/mlx-lm-lora)
- **Model type:** {model.args.model_type}
- **Author:** None

---

## 💡 Recommended System Prompt

```text
{system_prompt}
```
"""

new_readme_path = f"{new_model_name}/README.md"
with open(new_readme_path, "w") as new_readme_file:
    new_readme_file.write(readme_file)

# Upload it to HugginFace

In [ ]:
api = HfApi(token=hf_token)
create_repo(
  repo_id = f"{user_name}/{new_model_name}",
  repo_type="model",
  exist_ok=True,
  token=hf_token,
  private=True
)
api.upload_folder(
  folder_path=new_model_name,
  repo_id=f"{user_name}/{new_model_name}",
  token=hf_token,
  commit_message="Initial Commit"
)